In [13]:
import pickle
import os
import yaml
import sys
import torch
import torch_geometric
import numpy as np
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GCN2Conv as GC2
from torch_geometric.nn import GATv2Conv 
from copy import deepcopy

import torch
from torch.nn import Linear, Parameter
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch.nn import Sequential as Seq, Linear, ReLU, SiLU

from math import log
from typing import Optional, Tuple

import torch
from torch import Tensor
from torch.nn import Parameter
from torch_sparse import SparseTensor, matmul

from torch_geometric.nn.conv import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.typing import Adj, OptTensor

# from ..inits import glorot
from torch.nn.init import xavier_uniform_ as glorot


In [14]:
with open('config.yaml', 'r') as config_file:  
    config = yaml.safe_load(config_file) 
    
with open(config['protein_config_file'], 'r') as config_file:  
    protein_config = yaml.safe_load(config_file)   

In [15]:
interaction_corpus = pickle.load(open(config['interaction_voxel_graph_dir'] + "interaction_voxel_corpus.pkl", 'rb'))
og_corpus = deepcopy(interaction_corpus)

In [16]:
target_list = np.array(interaction_corpus[0])
# graph_list = np.array(interaction_corpus[1])

In [17]:
MAX_EDGE_WEIGHT = 15.286330223083496

interaction_corpus = deepcopy(og_corpus)

for g_idx in range(len(interaction_corpus[1])):
    updated_edge_index = deepcopy(interaction_corpus[1][g_idx].edge_index)
    updated_edge_attr = deepcopy(interaction_corpus[1][g_idx].edge_attr)
    updated_x = deepcopy(interaction_corpus[1][g_idx].x)
    
    for e_idx in range(updated_edge_attr.size(0)):
        updated_edge_attr[e_idx][-1] /= MAX_EDGE_WEIGHT
    
    self_edges = torch.arange(updated_x.size(0)).unsqueeze(0)
    self_edges = torch.cat((self_edges,self_edges),dim=0)
    
    updated_edge_index = torch.hstack((updated_edge_index, self_edges))
    updated_edge_attr = torch.hstack((updated_edge_attr, torch.zeros((updated_edge_attr.size(0), 1))))
    
    self_loop_features = torch.zeros((updated_x.size(0), updated_edge_attr.size(1)))
    self_loop_features[:,-1] = 1
    
    updated_edge_attr = torch.vstack((updated_edge_attr, self_loop_features))
    x_coords = updated_x[:,-3:]
    updated_x = updated_x[:,:-3]
    
    interaction_corpus[1][g_idx] = Data(x=updated_x, 
                                        x_coords=x_coords,
                                        edge_index=updated_edge_index,
                                        edge_attr=updated_edge_attr,
                                        y=interaction_corpus[1][g_idx].y)
    


KeyboardInterrupt



In [ ]:
# for g_idx in range(len(interaction_corpus[1])):
#     interaction_corpus[1][g_idx].edge_attr = torch.hstack((interaction_corpus[1][0].edge_attr,
#                                                        torch.zeros((len(interaction_corpus[1][0].edge_attr)),1)))

In [ ]:
loader = DataLoader(interaction_corpus[1], batch_size=64, shuffle=True)

In [ ]:
class_count = [0 for x in protein_config['interaction_labels']]

for batch in loader:
    for class_idx in batch.y:
        class_count[class_idx] += 1
        
example_count = sum(class_count)
class_max = max(class_count)
class_weights = torch.tensor([1-x/example_count for x in class_count])
# class_weights = torch.tensor([1/(x/class_max) for x in class_count])
class_weights = torch.tensor([class_max/x for x in class_count])
class_weights[4] = 60
        
print(class_weights)

In [25]:
# pdist = torch.nn.PairwiseDistance(p=2) 

# class GC2Conv(MessagePassing):
#     _cached_edge_index: Optional[Tuple[Tensor, Tensor]]
#     _cached_adj_t: Optional[SparseTensor]

#     def __init__(self, channels: int, alpha: float, theta: float = None,
#                  layer: int = None, shared_weights: bool = True,
#                  cached: bool = False, add_self_loops: bool = True,
#                  normalize: bool = True, **kwargs):

#         kwargs.setdefault('aggr', 'add')
#         super().__init__(**kwargs)

#         self.channels = channels
#         self.alpha = alpha
#         self.beta = 1.
#         if theta is not None or layer is not None:
#             assert theta is not None and layer is not None
#             self.beta = log(theta / layer + 1)
#         self.cached = cached
#         self.normalize = normalize
#         self.add_self_loops = add_self_loops

#         self._cached_edge_index = None
#         self._cached_adj_t = None

#         self.weight1 = Parameter(torch.Tensor(channels, channels))

#         if shared_weights:
#             self.register_parameter('weight2', None)
#         else:
#             self.weight2 = Parameter(torch.Tensor(channels, channels))
            
#         MESSAGE_DIMS = channels*2+33
#         self.message_conv = Seq(Linear(MESSAGE_DIMS, MESSAGE_DIMS,
#                              SiLU(),
#                              Linear(MESSAGE_DIMS, channels), )

#         self.reset_parameters()
        
#     def reset_parameters(self):
#         self.weight1 = Parameter(glorot(torch.Tensor(self.channels, self.channels)))
#         self.weight2 = Parameter(glorot(torch.Tensor(self.channels, self.channels)))
        
#         self._cached_edge_index = None
#         self._cached_adj_t = None

#     def forward(self, x: Tensor, x_0: Tensor, edge_index: Adj,
#                 edge_weight: OptTensor=None, x_coords=None, edge_features=None) -> Tensor:

#         if self.normalize:
#             if isinstance(edge_index, Tensor):
#                 cache = self._cached_edge_index
#                 if cache is None:
#                     edge_index, edge_weight = gcn_norm(  # yapf: disable
#                         edge_index, edge_weight, x.size(self.node_dim), False,
#                         self.add_self_loops, self.flow, dtype=x.dtype)
#                     if self.cached:
#                         self._cached_edge_index = (edge_index, edge_weight)
#                 else:
#                     edge_index, edge_weight = cache[0], cache[1]

#             elif isinstance(edge_index, SparseTensor):
#                 cache = self._cached_adj_t
#                 if cache is None:
#                     edge_index = gcn_norm(  # yapf: disable
#                         edge_index, edge_weight, x.size(self.node_dim), False,
#                         self.add_self_loops, self.flow, dtype=x.dtype)
#                     if self.cached:
#                         self._cached_adj_t = edge_index
#                 else:
#                     edge_index = cache

#         # propagate_type: (x: Tensor, edge_weight: OptTensor)
#         x = self.propagate(edge_index, x_coords=x_coords, x=x, edge_weight=edge_weight, edge_features=edge_features, size=None)

#         x.mul_(1 - self.alpha)
#         x_0 = self.alpha * x_0[:x.size(0)]

#         if self.weight2 is None:
#             out = x.add_(x_0)
#             out = torch.addmm(out, out, self.weight1, beta=1. - self.beta,
#                               alpha=self.beta)
#         else:
#             out = torch.addmm(x, x, self.weight1, beta=1. - self.beta,
#                               alpha=self.beta)
#             out = out + torch.addmm(x_0, x_0, self.weight2,
#                                     beta=1. - self.beta, alpha=self.beta)

#         return out


#     def message(self, x_i: Tensor, x_j: Tensor, x_coords_i: Tensor, x_coords_j: Tensor, edge_weight: OptTensor, edge_features: Tensor) -> Tensor:
#         node_dist = pdist(x_coords_i, x_coords_j) / MAX_EDGE_WEIGHT
         
#         return x_j if edge_weight is None else node_dist.view(-1, 1) * x_j

#     def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
#         return matmul(adj_t, x, reduce=self.aggr)

#     def __repr__(self) -> str:
#         return (f'{self.__class__.__name__}({self.channels}, '
#                 f'alpha={self.alpha}, beta={self.beta})')
    
    
# hidden = 512 
# INTERACTION_TYPES = protein_config['interaction_labels']
# NODE_DIMS = 38 
# EDGE_DIMS = 9
# DUMMY_INDEX = protein_config['atom_labels'].index('DUMMY')

# class GCN(torch.nn.Module):
#     def __init__(self):
#         super().__init__()
#         self.linear1 = torch.nn.Linear(NODE_DIMS,hidden)
#         self.lin_edge = torch.nn.Linear(EDGE_DIMS-1, 32)
        
#         self.conv1 = GC2Conv(hidden, 0.2, add_self_loops=False)
#         self.conv2 = GC2Conv(hidden, 0.2, add_self_loops=False)
#         self.conv3 = GC2Conv(hidden, 0.2, add_self_loops=False)

#         self.linear2 = torch.nn.Linear(hidden, len(INTERACTION_TYPES))

#     def forward(self, data):
#         x, x_coords, edge_index, edge_features = data.x, data.x_coords, data.edge_index, torch.hstack((data.edge_attr[:,:-2], data.edge_attr[:,-1:]))

#         edge_features = self.lin_edge(edge_features)
#         x = self.linear1(x)

#         h = self.conv1(x, x, edge_index, x_coords=x_coords, edge_features=edge_features)
#         h = F.relu(h)

#         h = self.conv2(h, x, edge_index, x_coords=x_coords, edge_features=edge_features)
#         h = F.relu(h)

# #         h = self.conv2(h, x, edge_index, edge_weights)
# #         h = F.relu(h)
        
#         o = self.linear2(h)
#         return o

# model = GCN()

# for batch in loader:
#     print(model(batch))
#     break

torch.Size([9216, 32])


SystemExit: 

In [ ]:
pdist = torch.nn.PairwiseDistance(p=2) 

class GC2Conv(MessagePassing):
    _cached_edge_index: Optional[Tuple[Tensor, Tensor]]
    _cached_adj_t: Optional[SparseTensor]

    def __init__(self, channels: int, alpha: float, theta: float = None,
                 layer: int = None, shared_weights: bool = True,
                 cached: bool = False, add_self_loops: bool = True,
                 normalize: bool = True, **kwargs):

        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)

        self.channels = channels
        self.alpha = alpha
        self.beta = 1.
        if theta is not None or layer is not None:
            assert theta is not None and layer is not None
            self.beta = log(theta / layer + 1)
        self.cached = cached
        self.normalize = normalize
        self.add_self_loops = add_self_loops

        self._cached_edge_index = None
        self._cached_adj_t = None

        self.weight1 = Parameter(torch.Tensor(channels, channels))

        if shared_weights:
            self.register_parameter('weight2', None)
        else:
            self.weight2 = Parameter(torch.Tensor(channels, channels))
            
        MESSAGE_DIMS = channels*2+33
        self.message_conv = Seq(Linear(MESSAGE_DIMS, MESSAGE_DIMS,
                             SiLU(),
                             Linear(MESSAGE_DIMS, channels), )

        self.reset_parameters()
        
    def reset_parameters(self):
        self.weight1 = Parameter(glorot(torch.Tensor(self.channels, self.channels)))
        self.weight2 = Parameter(glorot(torch.Tensor(self.channels, self.channels)))
        
        self._cached_edge_index = None
        self._cached_adj_t = None

    def forward(self, x: Tensor, x_0: Tensor, edge_index: Adj,
                edge_weight: OptTensor=None, x_coords=None, edge_features=None) -> Tensor:

        if self.normalize:
            if isinstance(edge_index, Tensor):
                cache = self._cached_edge_index
                if cache is None:
                    edge_index, edge_weight = gcn_norm(  # yapf: disable
                        edge_index, edge_weight, x.size(self.node_dim), False,
                        self.add_self_loops, self.flow, dtype=x.dtype)
                    if self.cached:
                        self._cached_edge_index = (edge_index, edge_weight)
                else:
                    edge_index, edge_weight = cache[0], cache[1]

            elif isinstance(edge_index, SparseTensor):
                cache = self._cached_adj_t
                if cache is None:
                    edge_index = gcn_norm(  # yapf: disable
                        edge_index, edge_weight, x.size(self.node_dim), False,
                        self.add_self_loops, self.flow, dtype=x.dtype)
                    if self.cached:
                        self._cached_adj_t = edge_index
                else:
                    edge_index = cache

        # propagate_type: (x: Tensor, edge_weight: OptTensor)
        x = self.propagate(edge_index, x_coords=x_coords, x=x, edge_weight=edge_weight, edge_features=edge_features, size=None)

        x.mul_(1 - self.alpha)
        x_0 = self.alpha * x_0[:x.size(0)]

        if self.weight2 is None:
            out = x.add_(x_0)
            out = torch.addmm(out, out, self.weight1, beta=1. - self.beta,
                              alpha=self.beta)
        else:
            out = torch.addmm(x, x, self.weight1, beta=1. - self.beta,
                              alpha=self.beta)
            out = out + torch.addmm(x_0, x_0, self.weight2,
                                    beta=1. - self.beta, alpha=self.beta)

        return out


    def message(self, x_i: Tensor, x_j: Tensor, x_coords_i: Tensor, x_coords_j: Tensor, edge_weight: OptTensor, edge_features: Tensor) -> Tensor:
        node_dist = pdist(x_coords_i, x_coords_j) / MAX_EDGE_WEIGHT
         
        return x_j if edge_weight is None else node_dist.view(-1, 1) * x_j

    def message_and_aggregate(self, adj_t: SparseTensor, x: Tensor) -> Tensor:
        return matmul(adj_t, x, reduce=self.aggr)

    def __repr__(self) -> str:
        return (f'{self.__class__.__name__}({self.channels}, '
                f'alpha={self.alpha}, beta={self.beta})')
    
    
hidden = 512 
INTERACTION_TYPES = protein_config['interaction_labels']
NODE_DIMS = 38 
EDGE_DIMS = 9
DUMMY_INDEX = protein_config['atom_labels'].index('DUMMY')

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear1 = torch.nn.Linear(NODE_DIMS,hidden)
        self.lin_edge = torch.nn.Linear(EDGE_DIMS-1, 32)
        
        self.conv1 = GC2Conv(hidden, 0.2, add_self_loops=False)
        self.conv2 = GC2Conv(hidden, 0.2, add_self_loops=False)
        self.conv3 = GC2Conv(hidden, 0.2, add_self_loops=False)

        self.linear2 = torch.nn.Linear(hidden, len(INTERACTION_TYPES))

    def forward(self, data):
        x, x_coords, edge_index, edge_features = data.x, data.x_coords, data.edge_index, torch.hstack((data.edge_attr[:,:-2], data.edge_attr[:,-1:]))

        edge_features = self.lin_edge(edge_features)
        x = self.linear1(x)

        h = self.conv1(x, x, edge_index, x_coords=x_coords, edge_features=edge_features)
        h = F.relu(h)

        h = self.conv2(h, x, edge_index, x_coords=x_coords, edge_features=edge_features)
        h = F.relu(h)

#         h = self.conv2(h, x, edge_index, edge_weights)
#         h = F.relu(h)
        
        o = self.linear2(h)
        return o

model = GCN()

for batch in loader:
    print(model(batch))
    break

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_function = torch.nn.CrossEntropyLoss(weight=class_weights.to(device))

batch_idx = 0
for epoch in range(5000):
    avg_loss = []
    print("EPOCH %s" % epoch)
    
    for batch in loader:
        optimizer.zero_grad()
        batch = batch.to(device)
        dummy_indices = torch.where(batch.x[:,DUMMY_INDEX] == 1)
        
        out = model(batch)
        
        dummy_node_out = out[dummy_indices]
        loss = loss_function(dummy_node_out, batch.y)
        avg_loss.append(loss.item())
        loss.backward()
        optimizer.step()
        batch_idx += 1
    
    print("Average loss:", sum(avg_loss) / len(avg_loss))
    print(torch.argmax(dummy_node_out,dim=1)[:15])
    print(batch.y[:15])
    avg_loss = []